In [4]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys
import import_ipynb
import actual


logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

importing Jupyter notebook from actual.ipynb


In [5]:
%%capture
import os
import site


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from torch.utils import data

In [6]:
n_examples = 20
n_time_steps = 100
n_width = 11
n_height = 19
inputs = th.ones(n_examples, 1, n_time_steps, n_width * n_height)

In [7]:
device='cpu'

In [8]:
from __future__ import print_function
import torch
import torchvision

In [13]:
class OwnData(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, images, labels):
        """Initialization"""
        self.images = images
        self.labels = labels

    def __len__(self):
        """Denotes the total number of samples"""
        return len(self.images)

    def __getitem__(self, index):
        """Generates one sample of data"""

        # Load data and get label
        X = self.images[i]
        y = self.labels[i]

        return X, y


In [34]:
batch_size = 12

In [43]:
traindata = OwnData(torch.tensor(actual.images, dtype=torch.float).view(811, 1, 225, 11, 19), torch.tensor(actual.labels, dtype=torch.float))

trainloader = torch.utils.data.DataLoader(traindata, batch_size=batch_size,
                                          shuffle=True, num_workers=4)



testdata = OwnData(torch.tensor(actual.images_test, dtype=torch.float).view(160, 1, 225, 11, 19), torch.tensor(actual.labels_test, dtype=torch.float))

testloader = torch.utils.data.DataLoader(testdata, batch_size=batch_size,
                                        shuffle=False, num_workers=4)


In [127]:
n_filters = 15
n_filters_2 = 32
n_filters_3 = 64
n_filters_4 = 128
kernel_size = (21, 1, 1)

# continue: process time dimension several steps,
# then remove it by meaning across time
# -> n_examples x n_conv_out_filters x (width * height)
# -> transform back to:
# n_examples x n_conv_out_filters x width x height
# -> now it is a normal "image" problem

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(1, n_filters, kernel_size)
        self.conv2 = nn.Conv3d(n_filters, n_filters_2, kernel_size)
        self.conv3 = nn.Conv3d(n_filters_2, n_filters_3, kernel_size)
        # self.conv4 = nn.Conv2d(n_filters_3, n_filters_4, kernel_size)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(n_filters_3 * 209, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 20)
        self.fc4 = nn.Linear(20, 4)

    def forward(self, x):
        batch_size = x.size(0)
        # Max pooling over a (2, 2) window
        x = F.max_pool3d(F.relu(self.conv1(x)), (2, 1, 1))
        # If the size is a square you can only specify a single number
        x = F.max_pool3d(F.relu(self.conv2(x)), (2, 1, 1))
        x = F.max_pool3d(F.relu(self.conv3(x)), (2, 1, 1))
        x = torch.mean(x, -3)
        # print(x.size())
        # x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        # x = x.view(-1, self.num_flat_features(x))
        x = x.view(batch_size, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc4(self.fc3(x)))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
net.to(device)

Net(
  (conv1): Conv3d(1, 15, kernel_size=(21, 1, 1), stride=(1, 1, 1))
  (conv2): Conv3d(15, 32, kernel_size=(21, 1, 1), stride=(1, 1, 1))
  (conv3): Conv3d(32, 64, kernel_size=(21, 1, 1), stride=(1, 1, 1))
  (fc1): Linear(in_features=13376, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=4, bias=True)
)

In [128]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [24]:
d, l = traindata.images, traindata.labels
# print(e) # [19, 1, 11, 225]

In [19]:
i = 0


o = net(d[i].view(1, 1, 225, 11, 19).to(device))
print(o)

tensor([0.0720, 0.0000, 0.0875, 0.0000], grad_fn=<ReluBackward0>)


In [126]:
# loss = criterion(o, l[i].to(device))
# print(loss)

In [12]:
# nn.CrossEntropyLoss?
# nn.MSELoss

In [124]:
# criterion(torch.Tensor([0, 0.5, 0.5, 0]), torch.argmax(torch.Tensor([0, 1, 0, 0], dtype=torch.int64)))
t = torch.Tensor([[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0]])
t.argmax(1)


tensor([2, 0, 1])

In [129]:
for epoch in range(2):
    running_loss = 0.0
    for image, label in trainloader:
        # get the inputs; data is a list of [inputs, labels]
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(image)
        
        loss = criterion(outputs, label.argmax(1))
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        i += image.size(0)
        
        if i = 192:    # print every 200 batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 192))
            running_loss = 0.0
            i = 0

print('Finished Training')

Finished Training


In [150]:
return

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(traindata[0].shape[0]):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = traindata[0][i].to(device), traindata[1][i].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

SyntaxError: 'return' outside function (<ipython-input-150-b00fd3464342>, line 1)

In [149]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in testloader:
        outputs = net(image)
        predicted = torch.max(outputs.data)
        total += label.size(0)
        correct += max((outputs == label).sum().item() - 36, 0)

print(correct, total)
print('Accuracy of the network on the %d test images: %d %%' % (
    total, 100 * correct / total))



0 160
Accuracy of the network on the 160 test images: 0 %


In [37]:
correct = 0
total = 0
timages, tlabels = testdata
with torch.no_grad():
    for i in range(testdata[0].shape[0]):
        images, labels = timages[i].to(device), tlabels[i].to(device)
        outputs = net(images)
        predicted = torch.max(outputs.data)
        total += labels.size(0)
        correct += (outputs == labels).sum().item()
        # print(outputs, labels)

print('Accuracy of the network on the %d test images: %d %%' % (
    testdata[0].shape[0], 100 * correct / total))


RuntimeError: Expected 5-dimensional input for 5-dimensional weight 15 1 21 1, but got 4-dimensional input of size [225, 1, 11, 19] instead

In [71]:
outputs

tensor([0.3460, 0.4153, 0.2221, 0.0000], device='cuda:0')

In [23]:
inputs = d[0].view(1, 1, 225, 11, 19).to(device)
for name, module in net.named_children():
    print(name)
    inputs = module(inputs)
    print(inputs.shape)
    print()

conv1
torch.Size([1, 15, 205, 11, 19])

conv2
torch.Size([1, 32, 185, 11, 19])

conv3
torch.Size([1, 64, 165, 11, 19])

fc1


RuntimeError: size mismatch, m1: [116160 x 19], m2: [13376 x 500] at /opt/conda/conda-bld/pytorch_1556653114079/work/aten/src/THC/generic/THCTensorMathBlas.cu:268

In [36]:
for data, target in it:
    print(target)

Exception in callback BaseAsyncIOLoop._handle_events(33, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(33, 1)>
Traceback (most recent call last):
  File "/home/ced/miniconda3/envs/PyTorch/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "/home/ced/miniconda3/envs/PyTorch/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 138, in _handle_events
    handler_func(fileobj, events)
  File "/home/ced/miniconda3/envs/PyTorch/lib/python3.7/site-packages/zmq/eventloop/zmqstream.py", line 456, in _handle_events
    self._handle_recv()
  File "/home/ced/miniconda3/envs/PyTorch/lib/python3.7/site-packages/zmq/eventloop/zmqstream.py", line 486, in _handle_recv
    self._run_callback(callback, msg)
  File "/home/ced/miniconda3/envs/PyTorch/lib/python3.7/site-packages/zmq/eventloop/zmqstream.py", line 438, in _run_callback
    callback(*args, **kwargs)
  File "/home/ced/miniconda3/envs/PyTorch/lib/python3.7/site-packages/ipyk

KeyboardInterrupt: 

In [46]:
type(trainloader.dataset)

tuple

In [57]:
it.next()

RuntimeError: Traceback (most recent call last):
  File "/home/ced/miniconda3/envs/PyTorch/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ced/miniconda3/envs/PyTorch/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 43, in default_collate
    return torch.stack(batch, 0, out=out)
RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 3 and 5 at /opt/conda/conda-bld/pytorch_1556653114079/work/aten/src/TH/generic/THTensor.cpp:702
